In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
!which python

In [ ]:
# 1. Data Load
datos = pd.read_csv("../Data/Salario/Salary_Data.csv")

In [ ]:
#2. Data clean
datos.isna().sum()

In [ ]:
#Data recode

In [ ]:
# Data split
df_entrenamiento = datos.sample(frac=0.8, random_state=0)
df_pruebas = datos.drop(index=df_entrenamiento.index)

In [ ]:
# Data inspect
df_stats_trainning = df_entrenamiento.describe()

In [ ]:
# Data Split IN/OUT

output_df_entrenamiento = df_entrenamiento.pop("Salary")
input_df_entrenamiento = df_entrenamiento

output_df_pruebas = df_pruebas.pop("Salary")
input_df_pruebas = df_pruebas

In [ ]:
# Normalized Data

normed_df_entrenamiento = (input_df_entrenamiento - df_stats_trainning.loc["mean","YearsExperience"])/(df_stats_trainning.loc["std","YearsExperience"])
normed_df_entrenamiento.describe()

In [ ]:
# Model Build

modelo = keras.Sequential()
modelo.add(layers.Dense(1, activation="linear", input_shape=[len(normed_df_entrenamiento.keys()), ]))
#modelo.add(layers.Dense(5, activation="sigmoid"))
#modelo.add(layers.Dense(1, activation="linear"))

alg_entrenamiento = keras.optimizers.RMSprop(learning_rate=200)
modelo.compile(loss="mae", metrics=["mae", "mse"], optimizer=alg_entrenamiento)

In [ ]:
# Model preview
modelo.summary()


In [ ]:
# Model Probe
modelo.predict(normed_df_entrenamiento[1:3])


In [ ]:
history = modelo.fit(
    normed_df_entrenamiento,
    output_df_entrenamiento,
    epochs=1000,
    validation_split = 0.2,
    verbose=1
)

In [ ]:
# Model Validation
historico = pd.DataFrame(history.history)
historico["Epochs"] = history.epoch

In [ ]:
ax001 = sns.lineplot(data=historico, x="Epochs", y="loss", label = "Función de error")
ax001 = sns.lineplot(data=historico, x="Epochs", y="val_loss", label = "Función de error en Validación", ax = ax001)
sns.move_legend(ax001, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
t1 = pd.DataFrame({"YearsExperience":(datos.YearsExperience.values - datos.YearsExperience.mean())/datos.YearsExperience.std()})
datos["Estimated"] = modelo.predict(t1)
ax001 = sns.scatterplot(data=datos, x="YearsExperience", y = "Salary", label="Datos Originales")
ax001 = sns.lineplot(data=datos, x="YearsExperience", y = "Estimated", label="Datos Estimados", ax = ax001)
sns.move_legend(ax001, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
datos["Residuals"] = datos["Salary"]-datos["Estimated"]
ax002 = sns.scatterplot(data= datos, x="YearsExperience", y = "Residuals")

In [ ]:
import statsmodels.tsa.api as smt
smt.graphics.plot_acf(datos["Residuals"],lags=29,alpha=0.05,title="Análisis de Autocovarianza")